# FMDA Dictionary Tutorial

The purpose of this notebook is to demonstrate creating fmda dictionaries to be used for training ML models of fuel moisture. This notebook combines the techniques from other notebooks in this directory, so see `interpolation_tutorial` and `synopticpy_tutorial` for more information. This code will live in `wrfxpy` in the python module `build_fmda_dict.py`.

**Goals:** given a user input of a date range and latitude/longitude bounding box, return a dictionary with top-level keys for each RAWS station within the bounding box that has fuel moisture data, and then for each station subdictionaries of formatted static location information, RAWS goundlevel sensor data, and atmospheric data from HRRR interpolated to the station location.

This notebook will demonstrate retrieving RAWS data using `SynopticPy`, but within `wrfxpy` for older times this data is retrieved from a stash of saved fuel moisture data.

*Note:* this requires a formatted stash of geotiff files, which are bands extracted from HRRR grib files.

## User Inputs

Below we manually enter the user inputs to define the spatiotemporal frame for the data collection. Within `wrfxpy`, these arguments are entered from the command line and read within python as system arguments with `sys.argv[...]`. The arguments should be formatted as:

* `start`: (str) start time formatted as "YYYYMMDDHHmm"
* `end`: (str) end time formatted as "YYYYMMDDHHmm"
* `bbox`: (list) of format `[lonmin, latmin, lonmax, lonmin]` (mimicking format from `SynopticPy`)

In [ ]:
start = "202401010000" # Jan 1, 2024, midnight UTC
end = "202401010200" # Jan 31, 2024, 2am UTC
bbox = [-105, 37, -103, 39]

## Static Data Objects

Below are objects decalred at the start of `build_fmda_dict.py` and used throughout. They include a dataframe of HRRR data bands, determined from [HRRR documentation](https://www.nco.ncep.noaa.gov/pmb/products/hrrr/hrrr.t00z.wrfprsf00.grib2.shtml). Also, there is a file path string object `hrrrpath` which points to the stash of formatted geotiff files. For this tutorial, those data simply live in the same directory.

In [ ]:
band_df_hrrr = pd.DataFrame({
    'Band': [585, 616, 620, 628, 629, 661, 662, 663, 664],
    'hrrr_name': ['GUST', 'TMP', 'RH', 'PRATE', 'APCP',
                  'DSWRF', 'DLWRF', 'USWRF', 'ULWRF'],
    'descr': ['surface Wind Speed (Gust) [m/s]',
              '2 m Temperature [K]', 
              '2 m Relative Humidity [%]', 
              'surface Precip. Rate [kg/m^2/s]',
              'surface Total Precipitation [kg/m^2]',
              'surface Downward Short-Wave Radiation Flux [W/m^2]',
              'surface Downward Long-Wave Rad. Flux [W/m^2]',
              'surface Upward Short-Wave Radiation Flux [W/m^2]',
              'Upward Long-Wave Rad. Flux [W/m^2]']
})

hrrrpath = "~/geotiff_files" # path for atmospheric data stash